In [2]:
%load_ext autoreload
%autoreload 2

import os
import json
import pandas as pd

from logging import getLogger

from dotenv import load_dotenv
load_dotenv()

from core.utils.logging import setup_logging
from core.utils import CONFIGS_DIR
from core.config import ConfigManager

from core.models.llm import LLMModel
from core.models.lmv3.model import LMv3Model

from data.utils import get_dataset, repair_bio_labels, build_page_json
from data.filters import filter_schematisms


/Users/user/Projects/AI_Osrodek/configs
/Users/user/Projects/AI_Osrodek/configs
PyTorch version 2.7.0 available.
/Users/user/Projects/AI_Osrodek/configs
/Users/user/Projects/AI_Osrodek/configs


In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
setup_logging()
logger = getLogger(__name__)
logger.info("Logger has ben setup")

/Users/user/Projects/AI_Osrodek/configs
Logger has ben setup


In [5]:
config_manager = ConfigManager(configs_dir=CONFIGS_DIR)
logger.info(CONFIGS_DIR)
config_manager.get_available_configs()

/Users/user/Projects/AI_Osrodek/configs
Found config directory: llm
Found config directory: dataset
Found config directory: tests
Found config directory: lmv3
Found config directory: logging


{}

In [6]:
llm_config = config_manager.load_config(
    config_name="config",
    config_dir="llm"
    
)

lmv3_config = config_manager.load_config(
    config_name="config",
    config_dir="lmv3"
)


dataset_config = config_manager.load_config(
    config_name="config",
    config_dir="dataset"
)

streamed_dataset_config = config_manager.load_config(
    config_name="streamed_dataset",
    config_dir="dataset"
)



Loaded config: {'predictor': {'api_type': 'openrouter', 'template_dir': 'prompts'}, 'interfaces': {'lm_studio': {'model': 'gemma-3-27b-it-qat', 'structured_output': True, 'base_url': 'http://localhost:1234/v1', 'template_dir': 'prompts', 'api_kwargs': {'max_tokens': 4096, 'temperature': 0.1, 'top_p': 0.9}}, 'openrouter': {'model': 'mistralai/pixtral-large-2411', 'base_url': 'https://openrouter.ai/api/v1', 'api_key_env_var': 'OPENROUTER_API_KEY', 'structured_output': True, 'api_kwargs': {'max_tokens': 4096, 'temperature': 0.1, 'top_p': 0.9}}, 'openai': {'model': 'gemma-3-27b-it-qat', 'api_key_env_var': 'OPENAI_API_KEY', 'structured_output': True, 'api_kwargs': {'max_tokens': 4096, 'temperature': 0.1, 'top_p': 0.9}}, 'mistral': {'model': 'mistral-large-latest', 'api_key_env_var': 'MISTRAL_API_KEY', 'structured_output': True, 'template_dir': 'prompts', 'api_kwargs': {'max_tokens': 4096, 'temperature': 0.1, 'top_p': 0.9}}}}
Loaded config: {'model': {'checkpoint': 'microsoft/layoutlmv3-base

In [7]:
dataset = get_dataset(dataset_config)

EcclesialSchematisms.py:   0%|          | 0.00/3.28k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/503 [00:00<?, ?B/s]

Using the latest cached version of the module from /Users/user/.cache/huggingface/modules/datasets_modules/datasets/artpods56--EcclesialSchematisms/a724fb67b8d27b253099abcec772b48750d32437756298596b42ac20c9a96a1b (last modified on Thu Jun 26 16:28:14 2025) since it couldn't be found locally at artpods56/EcclesialSchematisms, or remotely on the Hugging Face Hub.


Using the latest cached version of the module from /Users/user/.cache/huggingface/modules/datasets_modules/datasets/artpods56--EcclesialSchematisms/a724fb67b8d27b253099abcec772b48750d32437756298596b42ac20c9a96a1b (last modified on Thu Jun 26 16:28:14 2025) since it couldn't be found locally at artpods56/EcclesialSchematisms, or remotely on the Hugging Face Hub.


train.jsonl:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/537 [00:00<?, ? examples/s]

Loading label annotations…
Loading label annotations…
Loading label annotations…
Loading label annotations…
Loading label annotations…
Loading label annotations…
Loading label annotations…
Loading label annotations…
Loaded 537 annotations.
Iterating /Users/user/.cache/huggingface/hub/datasets--artpods56--EcclesialSchematisms/snapshots/98d5ff364fa6ad6cb81926d6206490c125cbf6f0/images/images_part1.tar
Loaded 537 annotations.
Loaded 537 annotations.
Iterating /Users/user/.cache/huggingface/hub/datasets--artpods56--EcclesialSchematisms/snapshots/98d5ff364fa6ad6cb81926d6206490c125cbf6f0/images/images_part5.tar
Iterating /Users/user/.cache/huggingface/hub/datasets--artpods56--EcclesialSchematisms/snapshots/98d5ff364fa6ad6cb81926d6206490c125cbf6f0/images/images_part9.tar
Loaded 537 annotations.
Iterating /Users/user/.cache/huggingface/hub/datasets--artpods56--EcclesialSchematisms/snapshots/98d5ff364fa6ad6cb81926d6206490c125cbf6f0/images/images_part14.tar
Loaded 537 annotations.
Iterating /User

In [8]:



filtered_dataset = dataset.filter(filter_schematisms(["wloclawek_1872","wloclawek_1873","tarnow_1870", "chelmno_1871"]))


Filter:   0%|          | 0/537 [00:00<?, ? examples/s]

In [9]:
filtered_dataset

Dataset({
    features: ['image_pil', 'image', 'width', 'height', 'words', 'bboxes', 'labels', 'conf'],
    num_rows: 496
})

In [10]:
df = pd.read_csv("/Users/user/Projects/AI_Osrodek/data/csv/dane_hasla_with_filename.csv")

In [11]:
df

,id,dekanat,parafia,miejsce,typ_obiektu,wezwanie,material_typ,the_geom,diecezja,strona_p,strona_k,skany,wezwanie_par,faksymile,file_name
0,22682,Wieliczka,Brzezie,Brzezie,kościół parafialny,Mikołaj Biskup,dr,"MULTIPOLYGON (((1900.072765 1690.88829, 1900.0...",Tarnów,143,143,tarnow_1871,NaN,NaN,tarnow_1871_0143.jpg
1,28124,Rogowo,Niestronno,Niestronno,kościół parafialny,[brak_informacji],[brak_informacji],MULTIPOLYGON (((521.99899984742 2558.309000152...,Gniezno-Poznań,31,31,gniezno_1870,[brak_informacji],NaN,gniezno_1870_0031.jpg
2,28128,Rogowo,Świątkowo,Świątkowo,kościół parafialny,[brak_informacji],[brak_informacji],MULTIPOLYGON (((522.00699984742 2557.753000152...,Gniezno-Poznań,31,31,gniezno_1870,[brak_informacji],NaN,gniezno_1870_0031.jpg
3,10756,Wołkowysk,Krzemienica,Krzemienica,kaplica,Roch W,[brak_informacji],MULTIPOLYGON (((472.49839993285 756.0206000671...,Wilno,118,118,wilno_1938,NaN,0118.jpg,wilno_1938_0118.jpg
4,20664,Lidzbark-Pomezania,Górzno,Górzno,kościół parafialny,Św. Krzyż,[brak_informacji],MULTIPOLYGON (((365.8177499999899 1497.6750901...,Chełmno,72,72,chelmno_1939,NaN,0072.jpg,chelmno_1939_0072.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33033,28125,Rogowo,Palędzie,Palędzie,kościół parafialny,[brak_informacji],[brak_informacji],MULTIPOLYGON (((521.99899984742 2558.165000152...,Gniezno-Poznań,31,31,gniezno_1870,[brak_informacji],NaN,gniezno_1870_0031.jpg
33034,28126,Rogowo,Rogowo,Rogowo,kościół parafialny,[brak_informacji],[brak_informacji],MULTIPOLYGON (((522.00299984742 2558.025000152...,Gniezno-Poznań,31,31,gniezno_1870,[brak_informacji],NaN,gniezno_1870_0031.jpg
33035,14553,Lwów Miejski,Lwów - Wszyscy Święci,Lwów,kościół klasztorny,Wszyscy Św,[brak_informacji],"MULTIPOLYGON (((280.778 933.3150000000098, 280...",Lwów,35,35,lwow_1936,NaN,0035.jpg,lwow_1936_0035.jpg
33036,7062,Dubno,Dubno,Dubno,kościół parafialny,Jan N,mr,"MULTIPOLYGON (((160.435 429.0905, 160.435 429....",Łuck,41,42,luck_1938,NaN,0041.jpg,luck_1938_0041.jpg


In [12]:
llm_model = LLMModel(config=llm_config)
lmv3_model = LMv3Model(config=lmv3_config)

Initializing LLM client...
Testing connection to LLM client...
HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
Successfully connected to OpenAI client.
Id2label: {0: 'B-building_material', 1: 'B-deanery', 2: 'B-dedication', 3: 'B-page_number', 4: 'B-parish', 5: 'B-parish_dedication', 6: 'B-settlement_classification', 7: 'I-building_material', 8: 'I-deanery', 9: 'I-dedication', 10: 'I-parish', 11: 'I-parish_dedication', 12: 'O'}


In [15]:
with open("df_entries.json", "r") as f:
    df_entries = json.load(f)



In [19]:
df_entries

{'chelmno_1871_0000a.jpg': {'predictions': {'page_number': None,
   'deanery': None,
   'entries': []},
  'metrics': {'page_number': {'TP': 0,
    'FP': 0,
    'FN': 0,
    'precision': 0.0,
    'recall': 0.0},
   'deanery': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'parish': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'dedication': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'building_material': {'TP': 0,
    'FP': 0,
    'FN': 0,
    'precision': 0.0,
    'recall': 0.0}}},
 'chelmno_1871_0000b.jpg': {'predictions': {'page_number': None,
   'deanery': None,
   'entries': []},
  'metrics': {'page_number': {'TP': 0,
    'FP': 0,
    'FN': 0,
    'precision': 0.0,
    'recall': 0.0},
   'deanery': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'parish': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'dedication': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'building_m

In [18]:
import re

# customise *once* for your whole project
#   – tokens that mark an alternative / "former" name
#   – trailing abbreviations you want to drop
ALT_TOKENS   = {"vel", "ex", "il"}
TRAILING_ABB = re.compile(r",\s*(?:P\.?E\.?)$", re.I)

def normalize_parish(raw: str) -> str:
    """
    Strip stray punctuation, stop at "vel / ex / il …",
    fold repeated spaces, and Title-case the result.
    Keeps Polish diacritics intact.
    Also strips leading numbers and punctuation like "1. "
    """
    s = raw.strip()                           # whitespace at both ends
    s = re.sub(r"^\d+[.,;:\s]+", "", s)       # strip leading numbers and punctuation
    s = re.sub(r"[.,;:\s]+$", "", s)          # punctuation at the end
    s = s.lstrip("—–-")                       # long/short dashes at the start

    # cut everything from the first ALT_TOKEN onwards
    m = re.search(rf"\b({'|'.join(ALT_TOKENS)})\b", s, flags=re.I)
    if m:
        s = s[:m.start()].rstrip()

    # special case: ", P.E." and similar
    s = TRAILING_ABB.sub("", s)

    s = re.sub(r"\s+", " ", s)                # collapse multiple spaces
    if len(s) > 2:
        return s.title()                          # "łęki dólne" → "Łęki Dólne"
    else:
        return ""


# ── quick demo ─────────────────────────────────────────────────────────────────
parishes = [
    "1. Zakopane,",
    "2. Zawoja vel Skawica,",
    "3. Łapczyca,P.E.",
    "4. ex —",
    "5. ©",
]

clean = [normalize_parish(p) for p in parishes if normalize_parish(p)]
print(clean)
# ['Zakopane', 'Zawoja', 'Łapczyca']


['Zakopane', 'Zawoja', 'Łapczyca']


In [20]:
# ---------------------------
# building_materials.py
# ---------------------------

import re
from pathlib import Path
import csv

# ──────────────────────────────────────────────────────────────
# 1) Canonical sets (normalised, no punctuation, lowercase)
# ──────────────────────────────────────────────────────────────
WOOD = {
    "lig", "lign", "ligno", "lignea", "lig", "ligo", "lignum",  # core roots
    "exlig", "exlign", "exligno", "ec_lig",                     # compound / prefixed
}
BRICK = {
    "mur","mu", "muro", "murata", "exmuro", "m",
}
MIXED = {
    "mur_et_lign", "partim_mur", "partim_murata_partim_lign"
}
GENERIC = {
    "eduro_materiali", "duro_materiah",
}

VALID = WOOD | BRICK | MIXED | GENERIC

# ──────────────────────────────────────────────────────────────
# 2) Normalisation helpers
# ──────────────────────────────────────────────────────────────
PUNCT = re.compile(r"[.,;:\-_\s]+")

def normalise(raw: str | None) -> str:
    """
    Lower-case, strip whitespace, remove punctuation and diacritics.
    None -> '' so it fails validation.
    """
    if raw is None:
        return ""
    # quick & dirty latin ascii-only fold; tweak if you expect accents
    txt = raw.lower()
    txt = PUNCT.sub("", txt)
    return txt

def classify(raw: str | None) -> str:
    tok = normalise(raw)
    has_brick = any(x in tok for x in BRICK)
    has_wood = any(x in tok for x in WOOD)
    if has_brick and has_wood:
        return "mixed"
    if has_brick:
        return "brick"
    if has_wood:
        return "wood"
    if any(x in tok for x in GENERIC):
        return "generic"
    return "unknown"


In [23]:
# ---------------------------
# validate_dataset.py
# ---------------------------


def flag_bad_rows(dataset, report_path: str | Path | None = None):
    """Iterates over HF dataset (or list of dicts) and returns list of bad indices.

    Optionally writes a CSV report:  idx,page_id,bad_value
    """
    bad_samples = []
    csv_rows = []

    for idx, sample in enumerate(tqdm(dataset, desc="Scanning dataset")):
        words, bboxes, labels = sample["words"], sample["bboxes"], sample["labels"]
        repaired_labels = repair_bio_labels(labels)
        gt_json = build_page_json(words=words, bboxes=bboxes, labels=repaired_labels)

        for entity in gt_json["entries"]:
            bm_raw = entity.get("building_material")
            
            if bm_raw is None:
                continue  # might be None
            
            if classify(bm_raw) == "unknown":
                bad_samples.append(idx)
                csv_rows.append(
                    {
                        "idx": idx,
                        "page_id": sample["image"],
                        "bad_value": bm_raw,
                    }
                )
                # break if you only need flag once per page
                break

    # Optional report
    if report_path:
        with open(report_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["idx", "page_id", "bad_value"])
            writer.writeheader()
            writer.writerows(csv_rows)
        print(f"[INFO] Wrote {len(csv_rows)} flagged rows → {report_path}")

    print(f"[SUMMARY] {len(bad_samples)} / {len(dataset)} samples need review.")
    return bad_samples


def get_unique_values(dataset):
    unique_values = {}
    for idx, sample in enumerate(tqdm(dataset, desc="Scanning dataset")):
        words, bboxes, labels = sample["words"], sample["bboxes"], sample["labels"]
        repaired_labels = repair_bio_labels(labels)
        sample_json = build_page_json(words=words, bboxes=bboxes, labels=repaired_labels)
        for entity in sample_json["entries"]:
            for key, value in entity.items():
                # if key == "parish":
                #     normalized_value = normalize_parish(value)
                # else:
                #     normalized_value = normalise(value)

                if key not in unique_values:
                    unique_values[key] = set()
                else:
                    unique_values[key].add(value)
    return unique_values


from tqdm import tqdm

def get_dict_unique_values(dict_of_entries):
    unique_values = {}
    for file_name, entry_data in dict_of_entries.items():
        if entry_data["predictions"] is not None:
            for key, entries in entry_data["predictions"].items():
                if entries is not None:
                    if isinstance(entries, list):
                        for entry in entries:
                            if isinstance(entry, dict):
                                for key, value in entry.items():
                                    if key not in unique_values:
                                        unique_values[key] = set()
                                    unique_values[key].add(value)
    return unique_values


unique_values_df = get_dict_unique_values(df_entries)

In [21]:
sorted(unique_values_df["parish"])

In [22]:
df_entries

In [23]:
unique_values = get_unique_values(filtered_dataset)

['!',
 '& Rehwalde.',
 "'",
 "'Wysoka,",
 '*raudenz.',
 '-',
 '- Nagoszyn,',
 '-Ecclesiae',
 '.',
 '1 Brzesko,',
 '1.',
 '1. Częstochowa,',
 '1. Grodzieo,',
 '1. Ozastary,',
 '1. Petroków,',
 '1. Radziejów,',
 '1. Sieradz,',
 '1. Wioclawek,',
 '1. Włocławek,',
 '10. Dobryszyce,',
 '10. Koscieleo,',
 '10. Koseielec,',
 '10. Kowalewo,',
 '10. Krzywosądza,',
 '10. Kwiatkowice,',
 '10. Kłótno,',
 '10. Mykanów,',
 '11, Parzymiechy,',
 '11.',
 '11. Dworszewice,',
 '11. Królikowo,',
 '11. Lisków,',
 '11. Luhotyń,',
 '11. Marzenin,',
 '11.Wielkalgka.',
 '12. Gidle,',
 '12. Godziesze,',
 '12. Kaszewice,',
 '12. Kruszyn,',
 '12. Lądek,',
 '12. Malanów,',
 '12. Mikołajewice,',
 '12. Poczesna,',
 '13.',
 '13. Goliszew,',
 '13. Jedlno,',
 '13. Kociszew,',
 '13. Lubstówek,',
 '13. Morzysław,',
 '13. Mąkoszyn,',
 '13. Młodojewo,',
 '13. Pabijanice,',
 '13. Przyrów,',
 '13. Łowiczek,',
 '14. Góra,',
 '14. Krzepozów,',
 '14. Kłomnice,',
 '14. Małyń,',
 '14. Mysliborz,',
 '14. Mąkolno,',
 '14. Nieszawa,

In [75]:
unnormalized_unique_values = get_unique_values(filtered_dataset)

{'chelmno_1871_0000a.jpg': {'predictions': {'page_number': None,
   'deanery': None,
   'entries': []},
  'metrics': {'page_number': {'TP': 0,
    'FP': 0,
    'FN': 0,
    'precision': 0.0,
    'recall': 0.0},
   'deanery': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'parish': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'dedication': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'building_material': {'TP': 0,
    'FP': 0,
    'FN': 0,
    'precision': 0.0,
    'recall': 0.0}}},
 'chelmno_1871_0000b.jpg': {'predictions': {'page_number': None,
   'deanery': None,
   'entries': []},
  'metrics': {'page_number': {'TP': 0,
    'FP': 0,
    'FN': 0,
    'precision': 0.0,
    'recall': 0.0},
   'deanery': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'parish': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'dedication': {'TP': 0, 'FP': 0, 'FN': 0, 'precision': 0.0, 'recall': 0.0},
   'building_m

In [21]:
df.columns


Scanning dataset: 100%|██████████| 496/496 [00:27<00:00, 18.01it/s]


In [26]:
from thefuzz import fuzz


fuzz.partial_ratio("Łęki", "Łęki Dólne")

Scanning dataset: 100%|██████████| 496/496 [00:27<00:00, 17.79it/s]


In [40]:
import unicodedata
import re

def normalize_text(text: str) -> str:
    """
    Normalize text by:
    1. Converting to lowercase
    2. Removing diacritics/accents
    3. Removing punctuation
    4. Stripping whitespace
    """
    if not text:
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove diacritics/accents
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    
    # Remove punctuation and extra whitespace
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    
    return text

# Test the normalization with relevant examples from the schematisms
test_cases = [
    "Płużnitz",
    "Pluznitz.",
    "Ryńsk",
    "Neudorf",
    "St. Anna",
    "Góra św. Małgorzaty"
]

print("Normalization examples:")
for text in test_cases:
    print(f"{text:20} -> {normalize_text(text)}")



Index(['id', 'dekanat', 'parafia', 'miejsce', 'typ_obiektu', 'wezwanie',
       'material_typ', 'the_geom', 'diecezja', 'strona_p', 'strona_k', 'skany',
       'wezwanie_par', 'faksymile', 'file_name'],
      dtype='object')

In [29]:
from thefuzz import fuzz, process
import json
scorer = fuzz.partial_ratio
fuzzy_threshold = 80
import logging
logger.setLevel(logging.WARNING)

df_entries = {}

saints_mapping = []
building_materials_mapping = []

for idx, sample in enumerate(tqdm(filtered_dataset, desc="Scanning dataset")):
    file_name = sample["image"]

    words, bboxes, labels = sample["words"], sample["bboxes"], sample["labels"]
    repaired_labels = repair_bio_labels(labels)
    sample_json = build_page_json(words=words, bboxes=bboxes, labels=repaired_labels)   

    if len(sample_json["entries"]) == 0:
        continue

    # display(sample["image_pil"].resize((400, 600)))
    df_sample = df[df["file_name"] == file_name][["parafia", "wezwanie", "material_typ"]]
    database_json = df_sample.to_dict()
    entries = [] 
    for parish, dedication, material_type in zip(database_json["parafia"].values(), database_json["wezwanie"].values(), database_json["material_typ"].values()):
        df_entry = {
            "parish": parish,
            "dedication": dedication,
            "building_material": material_type
        }
        entries.append(df_entry)


    constructed_json = {"page_number": None, "deanery": None, "entries": entries}

    # sort by the parish key both constructed and sample json
    constructed_json["entries"] = sorted(constructed_json["entries"], key=lambda x: x["parish"])
    sample_json["entries"] = sorted(sample_json["entries"], key=lambda x: x["parish"])



    unmatched_pred = [normalize_text(e['parish']) for e in constructed_json["entries"]]
    unmatched_copy = unmatched_pred.copy()
    
    logger.info(f"[STATUS] dataset parishes {[e['parish'] for e in sample_json["entries"]]}")
    logger.info(f"[STATUS] database parishes {[e['parish'] for e in constructed_json["entries"]]}")
    for ground_entry in sample_json["entries"]:
        ground_parish = normalize_text(ground_entry['parish'])

        logger.info(f"[EVALUATING] searching for: {ground_parish}")
        logger.info(f"[STATUS] available parishes: {unmatched_pred}")
        match = process.extractOne(ground_parish, unmatched_pred, scorer=scorer, score_cutoff=fuzzy_threshold)
        
        if not match:
            logger.info(f"[NO MATCH] No match found for {ground_parish}")
            continue
        else:
            best_match, score = match
            idx = unmatched_copy.index(best_match)
            if ground_entry["dedication"] is not None:
                saints_mapping.append((ground_entry["dedication"], constructed_json["entries"][idx]["dedication"]))

            if ground_entry["building_material"] is not None:
                building_materials_mapping.append((ground_entry["building_material"], constructed_json["entries"][idx]["building_material"]))

            unmatched_copy.remove(best_match)


            logger.info(f"[MATCH] {ground_parish} -> {best_match} with score {score}")
            unmatched_pred.remove(best_match)
      

    if len(unmatched_pred) == 0:
        logger.info(f"\t[SUCCESS] All parishes matched for {file_name}")
    else:
        continue
        logger.warning(f"\t[FAILURE] {len(unmatched_pred)} parishes not matched for {file_name}")
        logger.warning(f"\t[FAILURE] unmatched parishes: {unmatched_pred}")
        logger.warning(f"\t[FAILURE] database parishes: {[e['parish'] for e in sample_json['entries']]}")
        logger.warning(f"\t[FAILURE] dataset parishes: {[e['parish'] for e in constructed_json['entries']]}")
        # logger.warning(f"\t[FAILURE] database json: {json.dumps(database_json, indent=4, ensure_ascii=False)}")
        # display(sample["image_pil"].resize((400, 600)))
    # for df_entry, sample_entry in zip(constructed_json["entries"], sample_json["entries"]):
    #     print(f"Parish: {df_entry['parish']} - {sample_entry['parish']}")
    #     print(fuzz.partial_ratio(df_entry["parish"], sample_entry["parish"]))

    # logger.info(f"Processing {file_name}")
    # logger.info(f"Sample JSON: {constructed_json['entries']}")
    # logger.info(f"Predic JSON: {sample_json['entries']}")
    

    df_entries[file_name] = entries

100

In [36]:
saints_mapping = []
building_materials_mapping = []

for idx, sample in enumerate(tqdm(filtered_dataset, desc="Scanning dataset")):
    file_name = sample["image"]
    
    words, bboxes, labels = sample["words"], sample["bboxes"], sample["labels"]
    repaired_labels = repair_bio_labels(labels)
    sample_json = build_page_json(words=words, bboxes=bboxes, labels=repaired_labels)   

    if len(sample_json["entries"]) == 0:
        continue

    # display(sample["image_pil"].resize((400, 600)))
    df_sample = df[df["file_name"] == file_name][["parafia", "wezwanie", "material_typ"]]
    database_json = df_sample.to_dict()
    entries = [] 
    for parish, dedication, material_type in zip(database_json["parafia"].values(), database_json["wezwanie"].values(), database_json["material_typ"].values()):
        df_entry = {
            "parish": parish,
            "dedication": dedication,
            "building_material": material_type
        }
        entries.append(df_entry)


    constructed_json = {"page_number": None, "deanery": None, "entries": entries}
    normalized_to_original = {}
    for i, entry in enumerate(constructed_json["entries"]):
        normalized_name = normalize_text(entry['parish'])
        normalized_to_original[normalized_name] = i
    
    unmatched_pred = list(normalized_to_original.keys())
    
    for ground_entry in sample_json["entries"]:
        ground_parish = normalize_text(ground_entry['parish'])
        
        match = process.extractOne(ground_parish, unmatched_pred, scorer=scorer, score_cutoff=fuzzy_threshold)
        
        if not match:
            logger.info(f"[NO MATCH] No match found for {ground_parish}")
            continue
        
        best_match, score = match
        original_idx = normalized_to_original[best_match]  # Get correct original index
        
        if ground_entry["dedication"] is not None:
            saints_mapping.append((ground_entry["dedication"], constructed_json["entries"][original_idx]["dedication"]))
        
        if ground_entry["building_material"] is not None:
            building_materials_mapping.append((ground_entry["building_material"], constructed_json["entries"][original_idx]["building_material"]))
        
        unmatched_pred.remove(best_match)
        logger.info(f"[MATCH] {ground_parish} -> {best_match} with score {score}")
    
    # Process all files, not just perfect matches
    if len(unmatched_pred) == 0:
        logger.info(f"\t[SUCCESS] All parishes matched for {file_name}")
    else:
        logger.warning(f"\t[PARTIAL] {len(unmatched_pred)} parishes not matched for {file_name}")

Normalization examples:
Płużnitz             -> puznitz
Pluznitz.            -> pluznitz
Ryńsk                -> rynsk
Neudorf              -> neudorf
St. Anna             -> st anna
Góra św. Małgorzaty  -> gora sw magorzaty


In [70]:
set(building_materials_mapping)

Scanning dataset:   5%|▍         | 24/496 [00:00<00:13, 33.92it/s]

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


Scanning dataset:  69%|██████▉   | 342/496 [00:14<00:06, 23.11it/s]

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


Scanning dataset:  74%|███████▍  | 366/496 [00:15<00:05, 22.75it/s]

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


Scanning dataset: 100%|██████████| 496/496 [00:21<00:00, 23.53it/s]


In [108]:
saints_mapping

Scanning dataset:   3%|▎         | 14/496 [00:00<00:16, 29.16it/s]

	[PARTIAL] 1 parishes not matched for chelmno_1871_0011.jpg


Scanning dataset:   3%|▎         | 17/496 [00:00<00:17, 27.09it/s]

	[PARTIAL] 1 parishes not matched for chelmno_1871_0015.jpg


Scanning dataset:   4%|▍         | 20/496 [00:00<00:17, 27.34it/s]

...

Scanning dataset:  83%|████████▎ | 413/496 [00:20<00:03, 21.54it/s]

	[PARTIAL] 1 parishes not matched for wloclawek_1872_0096.jpg


Scanning dataset:  99%|█████████▉| 492/496 [00:23<00:00, 25.16it/s]

	[PARTIAL] 1 parishes not matched for wloclawek_1873_0054.jpg


Scanning dataset: 100%|██████████| 496/496 [00:23<00:00, 21.08it/s]


In [78]:
# load csv in src/experiments/notebooks/wezwania_202506271110.csv as json 

import pandas as pd

# Load the CSV file
saints_df = pd.read_csv('wezwania_202506271110.csv', usecols=["id", "wezwanie"])

# Convert to list of dictionaries
saints_json = saints_df.to_dict(orient='records')


mapping = {}

for entry in saints_json:
    mapping[entry["wezwanie"]] = entry["id"]

with open("database_dedication_mapping.json", "w") as f:
    json.dump(mapping, f, indent=4, ensure_ascii=False)

{('Cruc,', '[brak_informacji]'),
 ('EC.lig.', 'dr'),
 ('ex lig,', 'dr'),
 ('ex lig.', 'dr'),
 ('ex lign.', 'dr'),
 ('ex ligno', 'dr'),
 ('ex ligno.', 'dr'),
 ('ex mu- ro', 'mr'),
 ('ex muro', 'mr'),
 ('ex muro;', 'mr'),
 ('igue', '[brak_informacji]'),
 ('lg.', 'dr'),
 ('lie.', 'dr'),
 ('lig', 'dr'),
 ('lig,', 'dr'),
 ('lig.', 'dr'),
 ('lig.n.', 'dr'),
 ('liga.', 'dr'),
 ('lign', 'dr'),
 ('lign,', 'dr'),
 ('lign.', 'dr'),
 ('lign.', 'dr-mr'),
 ('lign.', 'mr'),
 ('lignea', 'dr'),
 ('lignea,', 'dr'),
 ('lignea,', 'dr-mr'),
 ('lignea.', 'dr'),
 ('ligno', 'dr'),
 ('ligo.', 'dr'),
 ('lis.', 'dr'),
 ('liv.', 'dr'),
 ('liz.', 'dr'),
 ('m.', 'mr'),
 ('mor.', 'mr'),
 ('mu-', 'mr'),
 ('mu- ro', 'mr'),
 ('mu.', 'mr'),
 ('mur,', 'mr'),
 ('mur.', 'dr'),
 ('mur.', 'mr'),
 ('mur.', 'mr?'),
 ('mur. a.', 'mr'),
 ('mur. et lign.', 'dr-mr'),
 ('murata', 'mr'),
 ('murata partim lign,', 'dr-mr'),
 ('murata.', 'mr'),
 ('muro', 'mr'),
 ('muro.', 'mr'),
 ('partim lign. partim mur.', 'dr-mr')}

In [109]:
saints_json

[('St. Nicolaum.', '[brak_informacji]'),
 ('St. Mariam.', 'Najświętsza Maria Panna'),
 ('Nativ. B. M. V.', 'Narodzenie Najświętszej Maryi Panny'),
 ('T. E. S. Adalbertus E. M,', 'Wojciech Biskup Męczennik'),
 ('T. E. OO. SS,', 'Wszyscy Święci'),
 ('Assumptionis B. M. V.', 'Najświętsza Maryja Panna Wniebowzięta'),
 ('S. Josephi Sponsi B. M. V.', 'Józef Oblubieniec Najświętszej Maryi Panny'),
 ('T. E. S. Casi- miri Ducis.', 'Kazimierz Królewicz'),
 ('T. E. S. Maria Magdalena.', 'Św. Duch, Anna'),
 ('Decollatio S. Joannis Bapt.', 'Ścięcie Jana Chrzciciela'),
 ('Omnium Sanctorum.', 'Wszyscy Św.'),
 ('SS. Stanislaus et Adalbertus.',
  'Stanisław Biskup Męczennik, Wojciech Biskup Męczennik'),
 ('T. E. S. Anna.', 'Anna'),
 ('— T. E. 00, SS.', 'Wszyscy Św.'),
 ('Assumpt. B. M. V.', 'Najświętsza Maryja Panna Wniebowzięta'),
 ('S. Bartholomaeus Ар.', 'Bartłomiej Apostoł'),
 ('S. Marcus Evang.', 'Marek Ewangelista'),
 ('E. S. Anna Mater B. M.', 'Anna Matka Najświętszej Maryi Panny'),
 ('SS. Apost

In [103]:
df_unique = get_dict_unique_values(df_entries)

In [94]:
len(df_unique["parish"])

[{'id': 1, 'wezwanie': '10Tys MM'},
 {'id': 2, 'wezwanie': '10Tys MM, Katarzyna, Dorota PPMM'},
 {'id': 3, 'wezwanie': '10Tys Rycerzy MM'},
 {'id': 4, 'wezwanie': '10TysRycerzy MM'},
 {'id': 5, 'wezwanie': '10TysRycerzy MM, Jakub W Ap, Anna'},
 {'id': 6, 'wezwanie': '11Tys PPMM'},
 {'id': 7, 'wezwanie': 'Agata PM'},
 {'id': 8, 'wezwanie': 'Agnieszka '},
 {'id': 9, 'wezwanie': 'Aleksander'},
 {'id': 10, 'wezwanie': 'Aleksy W'},
 {'id': 1101, 'wezwanie': 'Aleksy W, Benedykt O, Jadwiga Wd'},
 {'id': 11, 'wezwanie': 'Aleksy W, Jadwiga Wd'},
 {'id': 1117, 'wezwanie': 'Alfons Rodriguez'},
 {'id': 12, 'wezwanie': 'Alojzy'},
 {'id': 13, 'wezwanie': 'Andrzej Ap'},
 {'id': 14, 'wezwanie': 'Andrzej Ap, Anna'},
 {'id': 15, 'wezwanie': 'Andrzej Ap, Dorota PM'},
 {'id': 16, 'wezwanie': 'Andrzej Ap, Jan Ch'},
 {'id': 17, 'wezwanie': 'Andrzej Ap, Katarzyna PM'},
 {'id': 18, 'wezwanie': 'Andrzej Ap, Leonard O'},
 {'id': 19, 'wezwanie': 'Andrzej Ap, Małgorzata PM'},
 {'id': 20, 'wezwanie': 'Andrzej Ap, 

In [60]:
bad_samples = flag_bad_rows(filtered_dataset, report_path="bad_samples.csv")